# Nhanh Bills - Local Testing

Notebook này cho phép chạy Nhanh Bills pipeline từ local, không cần gọi Cloud Run.

**⚠️ QUAN TRỌNG - Đồng bộ với CloudRun:**
- Notebook này import trực tiếp từ code chính (`src/features/nhanh/bills/`)
- Khi code trên CloudRun thay đổi, notebook tự động sử dụng code mới
- Không cần copy/paste code - chỉ cần chạy lại notebook sau khi code thay đổi

**Flow mới (Flatten trong Python, Load trực tiếp vào Fact Tables):**
1. Extract: Lấy dữ liệu bills từ Nhanh API (raw nested structures)
2. Flatten + Load: Flatten nested structures trong Python → Upload GCS (backup) → Load trực tiếp vào Fact Tables (`nhanhVN.fact_sales_bills_v3_0`, `nhanhVN.fact_sales_bills_product_v3_0`)
3. ✅ Không cần Transform step nữa - flatten đã được làm trong loader

**Sử dụng khi:**
- Test pipeline trước khi deploy
- Debug issues
- Chạy sync thủ công cho date range cụ thể
- Test với credentials local


In [8]:
import os
import sys
from datetime import datetime, timedelta
from dotenv import load_dotenv

# Add project root to path
# Notebook is at: src/features/nhanh/bills/notebooks/
# Need to go up 5 levels: notebooks -> bills -> nhanh -> features -> src -> project_root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..', '..'))
sys.path.insert(0, project_root)

# Load environment variables từ project root
env_path = os.path.join(project_root, '.env')
load_dotenv(dotenv_path=env_path)

print(f"Project root: {project_root}")
print(f"Python path: {sys.path[0]}")
print(f"Loading .env from: {env_path}")
print(f".env file exists: {os.path.exists(env_path)}")


Project root: d:\Atino\extension\etl-api-bigquery
Python path: d:\Atino\extension\etl-api-bigquery
Loading .env from: d:\Atino\extension\etl-api-bigquery\.env
.env file exists: True


In [9]:
# Import các module từ code chính
# Lưu ý: Notebook import trực tiếp từ src/features/nhanh/bills/ nên luôn đồng bộ với code CloudRun
from src.features.nhanh.bills.components.extractor import BillExtractor
from src.features.nhanh.bills.components.loader import BillLoader
from src.features.nhanh.bills.pipeline import BillPipeline
from src.config import settings
import logging

# Cấu hình logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ Đã import tất cả modules thành công")


✅ Đã import tất cả modules thành công


## Cấu hình

Load cấu hình từ environment variables và credential files.

**Lưu ý về Credentials:**
- Nhanh API credentials được lấy từ Secret Manager hoặc environment variables
- BigQuery credentials: Sử dụng Application Default Credentials (ADC) hoặc service account key
- GCS bucket: Cấu hình trong `BRONZE_BUCKET` environment variable


In [10]:
# ⚠️ QUAN TRỌNG: Reload modules sau khi sửa code
# Chạy cell này trước khi chạy pipeline để đảm bảo code mới nhất được sử dụng
import importlib
import src.features.nhanh.bills.components.extractor
import src.features.nhanh.bills.components.loader
import src.features.nhanh.bills.pipeline

# Reload modules
importlib.reload(src.features.nhanh.bills.components.extractor)
importlib.reload(src.features.nhanh.bills.components.loader)
importlib.reload(src.features.nhanh.bills.pipeline)

# Re-import sau khi reload
from src.features.nhanh.bills.components.extractor import BillExtractor
from src.features.nhanh.bills.components.loader import BillLoader
from src.features.nhanh.bills.pipeline import BillPipeline

# Re-initialize components với code mới
extractor = BillExtractor()
loader = BillLoader()
pipeline = BillPipeline()

print("✅ Đã reload modules và khởi tạo lại components thành công")


✅ Đã reload modules và khởi tạo lại components thành công


In [11]:
# Kiểm tra cấu hình
print(f"GCP Project: {settings.gcp_project}")
print(f"GCP Region: {settings.gcp_region}")
print(f"Bronze Bucket: {settings.bronze_bucket}")
print(f"Bronze Dataset: {settings.bronze_dataset}")
print(f"Target Dataset: {settings.target_dataset}")
print(f"Nhanh API Base URL: {settings.nhanh_api_base_url}")

# Kiểm tra credentials (nếu có trong env)
nhanh_creds_available = (
    os.getenv("NHANH_APP_ID") or 
    os.getenv("NHANH_BUSINESS_ID") or 
    os.getenv("NHANH_ACCESS_TOKEN")
)
print(f"\nNhanh Credentials từ env: {'✅ Có' if nhanh_creds_available else '⚠️ Không có (sẽ dùng Secret Manager)'}")


GCP Project: sync-nhanhvn-project
GCP Region: asia-southeast1
Bronze Bucket: sync-nhanhvn-project
Bronze Dataset: bronze
Target Dataset: nhanhVN
Nhanh API Base URL: https://pos.open.nhanh.vn

Nhanh Credentials từ env: ✅ Có


## Khởi tạo Components

Khởi tạo extractor, loader và pipeline.


In [12]:
# Khởi tạo các components
extractor = BillExtractor()
loader = BillLoader()

# Khởi tạo pipeline
pipeline = BillPipeline()

print("✅ Đã khởi tạo tất cả components thành công")


✅ Đã khởi tạo tất cả components thành công


## Chạy Pipeline

Chạy pipeline với các options khác nhau.


### Option 4: Chạy với Date Range Tùy Chỉnh

Chạy pipeline với ngày cụ thể.


In [ ]:
# Tùy chọn: Chạy với date range tùy chỉnh
custom_from_date = datetime(2025, 9, 1)
custom_to_date = datetime(2025, 9, 30)

print("=" * 60)
print(f"Đang chạy Pipeline cho date range: {custom_from_date.strftime('%Y-%m-%d')} đến {custom_to_date.strftime('%Y-%m-%d')}")
print("=" * 60)

result = pipeline.run_extract_load(
    from_date=custom_from_date,
    to_date=custom_to_date,
    process_by_day=True
)

print("\n" + "=" * 60)
print("Kết quả Pipeline:")
print("=" * 60)
for key, value in result.items():
    print(f"{key}: {value}")
print("=" * 60)


2025-12-17 15:44:45,235 - src.features.nhanh.bills.pipeline - INFO - Starting Extract-Load pipeline for bills (Day-by-Day)
2025-12-17 15:44:45,235 - src.features.nhanh.bills.pipeline - INFO - Starting Extract-Load pipeline for bills (Day-by-Day)
2025-12-17 15:44:45,238 - src.features.nhanh.bills.pipeline - INFO - Processing day 1/30: 2025-09-01
2025-12-17 15:44:45,238 - src.features.nhanh.bills.pipeline - INFO - Processing day 1/30: 2025-09-01
2025-12-17 15:44:45,240 - src.shared.nhanh.client - INFO - Starting paginated fetch for /bill/list | {"endpoint": "/bill/list"}
2025-12-17 15:44:45,240 - src.shared.nhanh.client - INFO - Starting paginated fetch for /bill/list | {"endpoint": "/bill/list"}


Đang chạy Pipeline cho date range: 2025-09-01 đến 2025-09-30


2025-12-17 15:45:21,074 - src.shared.nhanh.client - WARNING - No data in page 102 | {"page": 102, "response_code": 1, "response_data": [], "full_response": {"code": 1, "data": []}}
2025-12-17 15:45:21,074 - src.shared.nhanh.client - WARNING - No data in page 102 | {"page": 102, "response_code": 1, "response_data": [], "full_response": {"code": 1, "data": []}}
2025-12-17 15:45:21,075 - src.shared.nhanh.client - INFO - Completed paginated fetch: 5038 total records | {"total_records": 5038}
2025-12-17 15:45:21,075 - src.shared.nhanh.client - INFO - Completed paginated fetch: 5038 total records | {"total_records": 5038}
2025-12-17 15:45:21,076 - src.features.nhanh.bills.components.extractor - INFO - Completed chunk 1: 5038 bills | {"chunk": 1, "bills_in_chunk": 5038, "total_bills": 5038}
2025-12-17 15:45:21,076 - src.features.nhanh.bills.components.extractor - INFO - Completed chunk 1: 5038 bills | {"chunk": 1, "bills_in_chunk": 5038, "total_bills": 5038}
2025-12-17 15:45:21,078 - src.feat

## Test Từng Component Riêng Lẻ

Test từng component riêng lẻ để debug.


### Test Extractor

Test extractor để lấy dữ liệu từ Nhanh API.


In [ ]:
# # Test Extractor - Extract bills và products
# test_from_date = datetime.now() - timedelta(days=7)
# test_to_date = datetime.now()

# print(f"Đang test Extractor cho date range: {test_from_date.strftime('%Y-%m-%d')} đến {test_to_date.strftime('%Y-%m-%d')}")

# bills, products = extractor.extract_with_products(
#     from_date=test_from_date,
#     to_date=test_to_date,
#     process_by_day=True
# )

# print(f"\nKết quả extract:")
# print(f"  - Bills: {len(bills)} records")
# print(f"  - Products: {len(products)} records")
# if bills:
#     print(f"  - Sample bill columns: {list(bills[0].keys())[:10]}")
# if products:
#     print(f"  - Sample product columns: {list(products[0].keys())[:10]}")


### Test Loader

Có 2 options:
1. **Load từ raw data**: Flatten nested structures, upload GCS, load vào Fact Tables
2. **Load từ GCS file có sẵn**: Load trực tiếp từ Parquet file đã flatten vào BigQuery (không cần extract/flatten lại)


In [ ]:


# # Option 2: Load từ raw data (cần flatten/upload lại)
# # Uncomment để test với raw nested data
# # test_date = date.today()
# # test_bills = [{
# #     "id": 12345,
# #     "depotId": 1,
# #     "date": "2025-01-15",
# #     "type": 2,
# #     "mode": 2,
# #     "customer": {"id": 100, "name": "Test Customer", "mobile": "0123456789", "address": "Test Address"},
# #     "sale": {"id": 50, "name": "Test Sale"},
# #     "created": {"id": 10, "name": "test@example.com"},
# #     "payment": {
# #         "amount": 1000000,
# #         "customerAmount": 1000000,
# #         "discount": 0,
# #         "points": 0,
# #         "cash": {"amount": 1000000},
# #         "transfer": {"amount": 0, "accountId": None},
# #         "credit": {"amount": 0}
# #     },
# #     "description": "Test bill"
# # }]
# # if test_bills:
# #     bills_path = loader.load_bills(data=test_bills, partition_date=test_date)
# #     print(f"✅ Bills flattened, uploaded to GCS: {bills_path}")


### Test Transformer (DEPRECATED)

⚠️ **DEPRECATED**: Transformer không còn cần thiết vì flatten đã được tích hợp vào loader.
Data được flatten và load trực tiếp vào Fact Tables trong loader.


In [ ]:
# # ⚠️ DEPRECATED: Transformer không còn cần thiết
# # Flatten đã được tích hợp vào loader, data được load trực tiếp vào Fact Tables
# # print("Đang test Transformer...")
# # print("⚠️ DEPRECATED: Transformer không còn cần thiết vì flatten đã được làm trong loader")
# # print("⚠️ Lưu ý: Cần có dữ liệu trong External Tables trước (nếu muốn test từ bronze raw)")

# # result = transformer.transform_flatten()

# # print(f"\nKết quả Transform:")
# # for key, value in result.items():
# #     print(f"  - {key}: {value}")
